5.1

In [10]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0553,  0.0518, -0.0556,  0.1170,  0.0451, -0.0277,  0.0269,  0.3343,
         -0.1412,  0.1650],
        [ 0.0223, -0.0314, -0.1861,  0.2239,  0.1616, -0.0850,  0.2400,  0.1275,
         -0.1105,  0.1800]], grad_fn=<AddmmBackward0>)

5.1.1

In [12]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [13]:
net = MLP()
net(X)

tensor([[-0.0828,  0.0598,  0.0174,  0.1662, -0.0421,  0.1867, -0.0934, -0.1720,
         -0.1198,  0.0550],
        [-0.0699,  0.0922, -0.0026,  0.0203, -0.0775,  0.1291, -0.0195, -0.1003,
         -0.0387,  0.1467]], grad_fn=<AddmmBackward0>)

5.1.2

In [14]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [15]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0749,  0.1234, -0.0970,  0.0220,  0.0600, -0.0559,  0.0361,  0.0952,
         -0.0615,  0.1298],
        [-0.0766,  0.1154, -0.0357, -0.0368,  0.0854,  0.1354, -0.0414,  0.0489,
         -0.0599,  0.1425]], grad_fn=<AddmmBackward0>)

5.1.3

In [30]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [31]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1457, grad_fn=<SumBackward0>)

In [32]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0315, grad_fn=<SumBackward0>)